In [1]:
%cd ..

/home/max/process/dist/app_noisemon


# Data wrangling

## Telegram export format

In [2]:
from typing import List

In [18]:
import regex
import json
from pathlib import Path
import pandas as pd

In [4]:
input_path = Path("./data/00-raw/full-time_trading_export.json")
input_path.exists()

True

In [5]:
data = json.loads(input_path.read_text())["messages"]

In [6]:
len(data)

80760

In [7]:
data[5]

{'id': 11,
 'type': 'message',
 'date': '2019-09-30T09:02:44',
 'edited': '2019-09-30T09:06:10',
 'from': '🔥Full-Time Trading',
 'from_id': 'channel1292964247',
 'file': '(File not included. Change data exporting settings to download.)',
 'thumbnail': '(File not included. Change data exporting settings to download.)',
 'media_type': 'animation',
 'mime_type': 'image/gif',
 'width': 1366,
 'height': 768,
 'text': '$ BABA продолжит падение после "подзарядки"  \nОбразовав паттерн "медвежий клин" и пробив его цена обрекла себя на падение согласно закону этого паттерна. Смотрите видео:'}

## Filter out non-textual

In [15]:
data = list(filter(lambda x: x["text"] , data))

In [16]:
data[56]

{'id': 83,
 'type': 'message',
 'date': '2019-10-05T14:55:58',
 'edited': '2019-10-05T17:45:25',
 'from': '🔥Full-Time Trading',
 'from_id': 'channel1292964247',
 'photo': '(File not included. Change data exporting settings to download.)',
 'width': 1280,
 'height': 591,
 'text': [{'type': 'hashtag', 'text': '#CMG'},
  ' инвест идея с триггером: прорыв цены в интрадей (Понедельник)\n\nОбязательные условия:\n1. Растущий SPY и хорошие новости\n2. Объем и моментум на момент прорыва большой\n\nПример как должно отработать смотрим на ',
  {'type': 'text_link',
   'text': 'видео в YouTube',
   'href': 'https://youtu.be/UYqklPxqSgQ'},
  '']}

In [13]:
len(data)

79085

## Glue chunked

In [10]:
def glue_chunks(chunks: List)-> str:
    buffer = []
    for chunk in chunks:
        if type(chunk) == str:
            buffer.append(chunk)
        else:
            buffer.append(chunk["text"])
    buffer = " ".join(buffer)
    buffer = regex.sub(r"\s{2, }", " ", buffer)
    return buffer

In [17]:
texts = []
for i in data:
    try:
        text = i['text']
        if type(text) == list:
            text = glue_chunks(text)
        texts.append(text)
    except:
        print(i)
        break

In [20]:
dataframe = pd.DataFrame({
    "text": texts,
    "timestamp": [x.get("edited") or x["date"] for x in data]
})

## Filter out messages not containing a ticker

In [14]:
import reticker

In [15]:
extractor = reticker.TickerExtractor()

In [16]:
with_ticker = list(filter(extractor.extract, texts))

In [17]:
len(with_ticker)

54101

In [18]:
with_ticker[0]

'$DIS  потенциально может свалиться в бездну, смотрим будет ли прорыв, если будет то тренд вниз на длительное время'

In [20]:
input_path.name.split(".")[0]

'full-time_trading_export'

In [22]:
output_name = input_path.name.split(".")[0] + "_with_ticker" + ".json"
output_path = Path("./data/01-filtered/") / output_name
with open(output_path, "w") as fout:
    json.dump(with_ticker, fout, ensure_ascii=True)

In [23]:
output_name = input_path.name.split(".")[0] + "_texts" + ".json"
output_path = Path("./data/01-filtered/") / output_name
with open(output_path, "w") as fout:
    json.dump(texts, fout, ensure_ascii=True)

In [21]:
output_name = input_path.name.split(".")[0] + "_with_dates" + ".json"
output_path = Path("./data/01-filtered/") / output_name
dataframe.to_json(output_path, orient="records")